Author: Kevin ALBERT  

Created: Oct 2020 

Based on: [git repo](https://github.com/lju-lazarevic/wine)

# environment

In [ ]:
# ! pip install py2neo pandas
# ! pip install pandas-profiling

In [ ]:
# ! pip install dtale

In [ ]:
import dtale

In [ ]:
d = dtale.show(prep, host="13.74.11.167", port="40000", ignore_duplicate=True, drop_index=True, reaper_on=False)

In [ ]:
# show all running instances
d.main_url()

In [ ]:
# stop webapp
d.kill()

In [1]:
! pip list |grep -i py2neo
! pip list |grep -i pandas

py2neo                        4.2.0
pandas                        1.1.3
pandas-profiling              2.9.0


py2neo is a client library and toolkit for working with Neo4j from within Python applications.  
It is well suited for Data Science workflows and has great integration with other Python Data Science tools.  
[py2neo docs](https://py2neo.org/v4/database.html)

In [2]:
from py2neo import Graph, Node, Relationship
import pandas as pd
from IPython.display import Javascript
import pandas_profiling as pp

In [3]:
neo_server = "13.74.11.167"
user = "neo4j"
passw = "digityser"
file = "winedata.csv"

In [4]:
graph = Graph(host=neo_server, auth=(user, passw))

In [5]:
# clean database
graph.delete_all()

query = """
CALL apoc.schema.assert({},{},true) YIELD label, key
RETURN *
"""
graph.run(query)
# alternative: shut down neo4j server, do a rm -rf data/graph.db and start up again

# dataReport

[link to the original dataset](https://www.kaggle.com/zynicide/wine-reviews/data)  
[link to the git repo dataset](https://github.com/lju-lazarevic/wine/tree/master/data)

In [6]:
# pre-cleaned dataset: deduplicated and cleaned twitter handles
datasetURL = "https://raw.githubusercontent.com/lju-lazarevic/wine/master/data/winemag-data-130k-v3.csv"
reportFile = "../../data/report/winemag_report.html"

In [7]:
df = pd.read_csv(datasetURL)

In [8]:
pp.ProfileReport(df=df.sample(frac=1),
                 minimal=True,
                 correlations={"cramers": {"calculate": False}}).to_file(reportFile)

In [9]:
# open the report (*.html)
display(Javascript('window.open("{url}");'.format(url=reportFile)))

<IPython.core.display.Javascript object>

# dataPrep
clean data prior to a load

In [10]:
prep = df[["winery", "province", "country"]].copy()

In [11]:
# replace nan
prep['winery'] = prep['winery'].fillna('No Winery')
prep['province'] = prep['province'].fillna('No Province')
prep['country'] = prep['country'].fillna('No Country')

In [12]:
# save file to /import
prep.to_csv("../../neo4j/import/"+file, sep=',', index=False)

# dataModel
[link to Arrows for data modelling](http://www.apcjones.com/arrows/#)

![Drag Racing](../../image/howto_graph/model2.jpg)

# dataLoading
Multistatement queries is only supported on neo4j browser or cypher shell.  
With py2neo you must run each statement sequentially. 
  
`MERGE` will take care of any duplicate values  
`p` temporary variable name  
`Province` entity name defined in data model  
`name` property name of entity  
`line.province` line is 1 record from dataset then select column province  
  
```cypher
MERGE (p: Province {name: (line.province)})
```

In [13]:
# check first 2 lines
! head -n 2 ../../neo4j/import/$file

winery,province,country
Nicosia,Sicily & Sardinia,Italy


In [14]:
# test data loading
query = """
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
WITH line
LIMIT 1
RETURN line
"""
data = graph.run(query)

In [15]:
next(data)

<Record line={'country': 'Italy', 'province': 'Sicily & Sardinia', 'winery': 'Nicosia'}>

In [16]:
# open neo4j dashboard
display(Javascript('window.open("{url}");'.format(url="http://"+neo_server+":7474")))

<IPython.core.display.Javascript object>

## createIndex

In [17]:
graph.run("""CREATE INDEX ON :Winery(name)""")
graph.run("""CREATE INDEX ON :Province(name)""")
graph.run("""CREATE INDEX ON :Country(name)""")

## createNodes

In [18]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MERGE (w: Winery {name: (line.winery)})
"""
graph.run(query)

query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MERGE (p: Province {name: (line.province)})
"""
graph.run(query)

query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MERGE (c: Country {name: (line.country)})
"""
graph.run(query)

## createRelations

In [19]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MATCH (w: Winery {name: trim(line.winery)})
MATCH (p: Province {name: trim(line.province)})
MATCH (c: Country {name: trim(line.country)})
MERGE (w)-[:FROM_PROVENCE]->(p)
MERGE (p)-[:PROVINCE_COUNTRY]->(c)
"""
graph.run(query)

## Which 10 countries have the most wineries ?
note: make sure to count only once each winery

In [20]:
%%time
query = """
MATCH (w:Winery)-[:FROM_PROVENCE]->(p:Province)-[:PROVINCE_COUNTRY]->(c:Country)
RETURN c.name AS Country, count(DISTINCT w) AS totalNrWineries
ORDER BY totalNrWineries DESC LIMIT 10
"""
graph.run(query).to_data_frame()

CPU times: user 4.41 ms, sys: 3.62 ms, total: 8.03 ms
Wall time: 1.02 s


,Country,totalNrWineries
0,US,5375
1,France,3864
2,Italy,2934
3,Spain,1435
4,Argentina,531
5,Australia,474
6,Portugal,430
7,Chile,317
8,New Zealand,300
9,South Africa,294


In [21]:
%%time
result = prep[["winery","country"]].groupby(['country'])['winery'].nunique()
result = result.rename_axis(['Country']).rename('totalNrWineries').sort_values(ascending=False).reset_index()
result.head(10)

CPU times: user 75.3 ms, sys: 413 µs, total: 75.8 ms
Wall time: 104 ms


,Country,totalNrWineries
0,US,5375
1,France,3864
2,Italy,2934
3,Spain,1435
4,Argentina,531
5,Australia,474
6,Portugal,430
7,Chile,317
8,New Zealand,300
9,South Africa,294


## Which wineries are across multiple provinces ?
alt: Which provinces are associated to each winery ?

In [22]:
%%time
query = """
MATCH (w:Winery)-[:FROM_PROVENCE]->(p:Province)
WITH w, COLLECT(p.name) AS Provinces, count(p) AS Total
RETURN w.name AS Winery, Provinces, Total
ORDER BY Total DESC LIMIT 10
"""
graph.run(query).to_data_frame()

CPU times: user 4.77 ms, sys: 0 ns, total: 4.77 ms
Wall time: 732 ms


,Winery,Provinces,Total
0,Undurraga,"[Colchagua Valley, Maule Valley, Maipo Valley,...",19
1,Concha y Toro,"[Colchagua Valley, Maule Valley, Maipo Valley,...",16
2,Santa Carolina,"[Colchagua Valley, Maule Valley, Maipo Valley,...",14
3,Kirkland Signature,"[Northern Spain, California, Mendoza Province,...",12
4,San Pedro,"[Northern Spain, Mendoza Province, Maule Valle...",12
5,Santa Rita,"[Colchagua Valley, Maipo Valley, Rapel Valley,...",11
6,Bacalhôa Wines of Portugal,"[Douro, Alentejano, Lisboa, Península de Setúb...",11
7,Casca Wines,"[Douro, Tejo, Alentejano, Vinho Verde, Lisboa,...",10
8,Tussock Jumper,"[Rheinhessen, California, Other, Colchagua Val...",10
9,Barton & Guestier,"[Beaujolais, Bordeaux, Burgundy, France Other,...",10


In [23]:
%%time
result = prep.groupby('winery').agg({'province':[lambda x: x.unique(), lambda x: x.nunique()]}).reset_index()
result.columns = ['Winery', 'Provinces', 'Total']
result = result.sort_values(by='Total',ascending=False).reset_index(drop=True)
result.head(10)

CPU times: user 4.58 s, sys: 28.7 ms, total: 4.6 s
Wall time: 4.79 s


,Winery,Provinces,Total
0,Undurraga,"[Maipo Valley, Leyda Valley, Chile, Cauquenes ...",19
1,Concha y Toro,"[Chile, Central Valley, Maipo Valley, Casablan...",16
2,Santa Carolina,"[Cachapoal Valley, Colchagua Valley, Casablanc...",14
3,San Pedro,"[Lontué Valley, Cachapoal Valley, Maipo Valley...",12
4,Kirkland Signature,"[California, Washington, Bordeaux, Rhône Valle...",12
5,Santa Rita,"[Leyda Valley, Central Valley, Maipo Valley, A...",11
6,Bacalhôa Wines of Portugal,"[Douro, Setubal, Península de Setúbal, Lisboa,...",11
7,Xavier Flouret,"[Central Valley, Bordeaux, Provence, Burgundy,...",10
8,Barton & Guestier,"[France Other, No Province, Bordeaux, Burgundy...",10
9,Echeverria,"[Central Valley, Maipo Valley, Curicó Valley, ...",10


# dataModel (expanded 1)
[link to Arrows for data modelling](http://www.apcjones.com/arrows/#)

![model3](../../image/howto_graph/model3.jpg)

![CALL db.schema.visualization](../../image/howto_graph/schema3.png)

# dataPrep (expanded 1)
clean data prior to a load

In [24]:
prep = df[["winery", "province", "country", "designation", "taster_name", "variety", "id", "title"]].copy()

In [25]:
prep['winery'] = prep['winery'].fillna('No Winery')
prep['province'] = prep['province'].fillna('No Province')
prep['country'] = prep['country'].fillna('No Country')

In [26]:
prep['designation'] = prep['designation'].fillna('No Designation')
prep['taster_name'] = prep['taster_name'].fillna('No Taster')
prep['variety'] = prep['variety'].fillna('No Variety')
prep['title'] = prep['title'].fillna('No Title')

In [27]:
# save file
prep.to_csv("../../neo4j/import/"+file, sep=',', index=False)

# dataLoading (expanded 1)
Multistatement queries is only supported on neo4j browser or cypher shell.  
With py2neo you must run each statement sequentially. 
  
`MERGE` will take care of any duplicate values  
`p` temporary variable name  
`Province` entity name defined in data model  
`name` property name of entity  
`line.province` line is 1 record from dataset then select column province  
  
```cypher
MERGE (p: Province {name: (line.province)})
```

In [28]:
# indexes for addition al data
graph.run("""CREATE INDEX ON :Wine(id)""")
graph.run("""CREATE INDEX ON :Taster(name)""")
graph.run("""CREATE INDEX ON :Variety(name)""")
graph.run("""CREATE INDEX ON :Designation(name)""")

In [29]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MERGE (d: Designation {name: (line.designation)})
"""
graph.run(query)

query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MERGE (t: Taster {name: (line.taster_name)})
"""
graph.run(query)

query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MERGE (v: Variety {name: (line.variety)})
"""
graph.run(query)

query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MERGE (c: Country {name: (line.country)})
MERGE (w: Wine {id: line.id, title: line.title})
"""
graph.run(query)

In [30]:
query = """
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///"""+file+"""' AS line FIELDTERMINATOR ','
MATCH (d: Designation {name: (line.designation)})
MATCH (t: Taster {name: (line.taster_name)})
MATCH (v: Variety {name: (line.variety)})
MATCH (w: Wine {id: (line.id)})
MATCH (win: Winery {name: (line.winery)})
MERGE (w)-[:FROM_WINERY]->(win)
MERGE (w)-[:HAS_VARIETY]->(v)
MERGE (t)-[:RATES_WINE]->(w)
MERGE (w)-[:HAS_DESIGNATION]->(d)
"""
graph.run(query)

# The top 10 most prolific wine tasters ?
note: not the unique amount but count the total amount tasted

In [31]:
%%time
query = """
MATCH (t:Taster)
WHERE t.name <> "No Taster"
//WITH t
MATCH (t)-[:RATES_WINE]->(w:Wine)-[:HAS_VARIETY]->(v:Variety)
WITH t, count(w) AS Total, COLLECT(DISTINCT v.name) AS Varieties
RETURN t.name AS Taster, Varieties, Total
ORDER BY Total DESC LIMIT 10
"""
graph.run(query).to_data_frame()

CPU times: user 5.1 ms, sys: 3.74 ms, total: 8.85 ms
Wall time: 2.63 s


,Taster,Varieties,Total
0,Roger Voss,"[Champagne Blend, Gamay, Grüner Veltliner, Pin...",23560
1,Michael Schachner,"[Sauvignon Blanc, Tempranillo, Red Blend, Verd...",14046
2,Kerin O’Keefe,"[Grillo, Sangiovese, Negroamaro, Moscato, Nebb...",9697
3,Paul Gregutt,"[Chardonnay, Cabernet Sauvignon, Pinot Noir, S...",8868
4,Virginie Boone,"[Sauvignon Blanc, Zinfandel, Merlot, Chardonna...",8708
5,Matt Kettmann,"[Chardonnay, Cabernet Franc, Syrah, Pinot Gris...",5730
6,Joe Czerwinski,"[Cabernet Merlot, Roussanne-Marsanne, Rhône-st...",4766
7,Sean P. Sullivan,"[Bordeaux-style Red Blend, Merlot, Riesling, S...",4461
8,Anna Lee C. Iijima,"[Cabernet Sauvignon, Riesling, Sauvignon Blanc...",4017
9,Jim Gordon,"[Pinot Noir, Dolcetto, Pinot Gris, Cabernet Sa...",3766


In [32]:
%%time
result = prep[prep.taster_name != "No Taster"]
result = result.groupby(['taster_name']).agg({'variety':[lambda x: list(x)], 'title':[lambda x: x.count()]}).reset_index()
result.columns = ['Taster', 'Varieties', 'Total']
result = result.sort_values(by='Total',ascending=False).reset_index(drop=True)
result.head(10)

CPU times: user 106 ms, sys: 3.38 ms, total: 109 ms
Wall time: 126 ms


,Taster,Varieties,Total
0,Roger Voss,"[Portuguese Red, Gewürztraminer, Pinot Gris, G...",23560
1,Michael Schachner,"[Tempranillo-Merlot, Malbec, Malbec, Tempranil...",14046
2,Kerin O’Keefe,"[White Blend, Frappato, Nerello Mascalese, Whi...",9697
3,Paul Gregutt,"[Pinot Gris, Pinot Noir, Pinot Noir, Pinot Noi...",8868
4,Virginie Boone,"[Cabernet Sauvignon, Cabernet Sauvignon, Pinot...",8708
5,Matt Kettmann,"[Chardonnay, Merlot, Sauvignon Blanc, Zinfande...",5730
6,Joe Czerwinski,"[Chardonnay, Rosé, Shiraz-Cabernet Sauvignon, ...",4766
7,Sean P. Sullivan,"[Malbec, Cabernet Franc, Bordeaux-style Red Bl...",4461
8,Anna Lee C. Iijima,"[Gewürztraminer, Riesling, Riesling, Riesling,...",4017
9,Jim Gordon,"[Red Blend, Cabernet Franc, White Blend, Grena...",3766


# How many wine varieties contain the word 'red' ?

In [33]:
%%time
query = """
MATCH (v:Variety)
WHERE tolower(v.name) CONTAINS 'red'
RETURN v.name
ORDER BY v.name
"""
graph.run(query).to_data_frame()

CPU times: user 3.32 ms, sys: 159 µs, total: 3.48 ms
Wall time: 105 ms


,v.name
0,Austrian Red Blend
1,Bordeaux-style Red Blend
2,Portuguese Red
3,Provence red blend
4,Red Blend
5,Rhône-style Red Blend
